In [82]:
import pandas as pd
import numpy as np
import statistics
import math
from datetime import timedelta
import matplotlib.pyplot as plt
import timeit

2020-10-19 17:40:46.306441 - user 9201 - retry 0 - time 0.07220077514648438 
        - returncode 0 
        - command ['/usr/bin/sbatch', '/tmp/condor_g_scratch.0x555e443cfe90.3058472/bl_129ykn']

In [41]:
# with open('../data/slurm_wrapper_ce5.log') as f:
#     data = ([line.split() for line in f if 'sbatch' in line and 'user 9204' in line])
# with open('../data/slurm_wrapper_ce6.log') as f:
#     data.extend([line.split() for line in f if 'sbatch' in line and 'user 9204' in line])
with open('../data/slurm_wrapper_ce5.log') as f:
    data = ([line.split() for line in f])
with open('../data/slurm_wrapper_ce6.log') as f:
    data.extend([line.split() for line in f])

In [42]:
sbatch = pd.DataFrame([[pd.to_datetime(x[0] + 'T' + x[1]), float(x[10]), int(x[13])] 
                       for x in data], columns=['BEGIN', 'DURATION', 'RETURNCODE'])

In [43]:
sbatch['RETURNCODE'].value_counts()

0      8218429
1      1296640
140      24987
255       7347
8           10
Name: RETURNCODE, dtype: int64

In [ ]:
# Check the duration of the process which ran before the one that timed out to see if we can
#  use this to flag the load if it is getting to be too much 
before = []
for index, row in sbatch.iterrows():
    if index and row['RETURNCODE'] == 1:
        before.append(sbatch.iloc[index - 1]['DURATION'])
print(statistics.mean(before))
print(statistics.stdev(before))
print(statistics.median(before))

In [ ]:
plt.boxplot(before)

In [ ]:
count = 0
for index, row in sbatch.iterrows():
    if index and row['RETURNCODE'] == 1 and sbatch.iloc[index - 1]['RETURNCODE'] == 0:
        count += 1
print(count)

In [ ]:
sbatch.describe()

In [45]:
dur = [math.modf(x) for x in sbatch['DURATION']]

In [46]:
# next(zip(*test[:10]))

In [47]:
td_input = [[x[1], x[0] * 1000000] for x in dur]
dur_td = [timedelta(seconds=x[0], microseconds=x[1]) for x in td_input]

In [48]:
sbatch['END'] = sbatch['BEGIN'].add(dur_td)

In [49]:

begin_load_df = sbatch[['BEGIN','RETURNCODE']].rename(columns={'BEGIN':'TIME'})
begin_load_df['process'] = 1
end_load_df = sbatch[['END','RETURNCODE']].rename(columns={'END':'TIME'})
end_load_df['process'] = -1
load_df = begin_load_df.append(end_load_df)
load_df = load_df.sort_values('TIME').reset_index()

In [50]:
load_df['running'] = np.cumsum(load_df['process'])

In [53]:
load_df['running'].value_counts()

1      3605737
0      2505577
2      1691299
3       969452
4       645490
        ...   
540          2
539          2
538          2
537          2
556          1
Name: running, Length: 557, dtype: int64

In [61]:
load_df[(load_df['running'] > 400) & (load_df['RETURNCODE'] == 0)]

,index,TIME,RETURNCODE,process,running
1559996,393611,2020-11-10 08:51:46.496072,0,1,401
1559997,5157480,2020-11-10 08:51:46.499066,0,1,402
1559998,393612,2020-11-10 08:51:46.518310,0,1,403
1559999,5157481,2020-11-10 08:51:46.536698,0,1,404
1560000,393613,2020-11-10 08:51:46.539404,0,1,405
...,...,...,...,...,...
18918351,9503566,2021-10-02 08:15:36.984302,0,1,440
18918352,9503482,2021-10-02 08:15:36.987820,0,-1,439
18918353,4726722,2021-10-02 08:15:36.993135,0,1,440
18918354,9503567,2021-10-02 08:15:36.998663,0,1,441


In [62]:
load_df[(load_df['running'] > 400) & (load_df['RETURNCODE'] == 1)]

,index,TIME,RETURNCODE,process,running
1560116,5157192,2020-11-10 08:51:50.935295,1,-1,407
1560266,393447,2020-11-10 08:51:54.099137,1,-1,433
1560272,393448,2020-11-10 08:51:54.147870,1,-1,437
1560330,393449,2020-11-10 08:51:54.845623,1,-1,429
1560344,393450,2020-11-10 08:51:54.965285,1,-1,417
...,...,...,...,...,...
18918392,9503317,2021-10-02 08:15:38.817160,1,-1,405
18918393,9503316,2021-10-02 08:15:38.824229,1,-1,404
18918394,9503310,2021-10-02 08:15:38.825354,1,-1,403
18918395,9503318,2021-10-02 08:15:38.830735,1,-1,402


In [79]:
    1 - (len(load_df[load_df['RETURNCODE'] == 1]) / len(load_df))

0.8641893882667483

In [80]:
len(load_df)

19094826

In [72]:
for i in range(6):
    print(len(load_df[(load_df['running'] > i * 100) & 
                      (load_df['running'] < (i + 1) * 100) & 
                      (load_df['RETURNCODE'] == 0)]) / 
          len(load_df[(load_df['running'] > i * 100) & 
                      (load_df['running'] < (i + 1) * 100) & 
                      (load_df['RETURNCODE'] == 1)]))

5.861656179654965
3.793390319053772
5.381062069648784
4.621005171727887
8.135135135135135
29.875


In [83]:
error_df = load_df[load_df['RETURNCODE'] == 1]